# EPSA Animated Plot

Let's work on creating our animated scatter plot.

The idea is to add a slider for the years to the scatter plot as well as 'stop/start' buttons together with smooth transitions of the dots resulting in a powerful animation showing the development of the EPSA over the years with respect to two indicators/variables. In the process we may add a third dimension in the form of the size of the dots corresponding to a different meassurement. Let's also don't forget the color of the dots will correspond to the category of the EPSA.

#### Plotly Offline

The use of plotly's servers is limited both by the hour and the day, based on number of requests, so it is better to work offline to generate the code and once you are satisfied with the results, commit them to the cloud. To do this is necessary to import a couple packages from **plotly.offline**.

In [2]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

print (__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)
# iplot([{"x": [1, 2, 3], "y": [3, 1, 6]}])

3.1.1


#### Collecting the Data

As always, we should start by getting our data ready for usage.
In this case, this process involves retrieving the data from our API and building the dataframes.

In [3]:
import requests
import pandas as pd

# API requests
reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')

# Builiding the Dataframes
reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)

# Merging the Tables
complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

#### Crating the Grid

Next, we should create the **Plotly Grid** that will feed our animated plot. A plotly grid consists of multiple columns with unique headers. It's basically a simple way to organize lists of values for usage. The structure of the headers is important, as these will be used to access the columns later on.

**Note**: As it turns out Grid only works with plotly online, also icreate_animations is not a part of plotly.offline. So for offline animations we have to use a work-around using pandas for feeding the data and use iplot instead.

In [93]:
import plotly.plotly as py
from plotly.grid_objs import Grid, Column

years = ['2014', '2015', '2016',]
categories = ['A', 'B', 'C', 'D']
plane_dimensions = ['v3', 'v22']
size_dimension = 'v22'


custom_colors = {
    'A': 'rgb(171, 99, 250)',
    'B': 'rgb(230, 99, 250)',
    'C': 'rgb(99, 110, 250)',
    'D': 'rgb(25, 211, 243)',
#     'Oceania': 'rgb(50, 170, 255)'
}

rdf = complete_reports_df
mdf = complete_measurements_df

columns = []

grid_data = {}

for year in years:
    for category in categories:
        filtered_rdf = rdf[(rdf.year == int(year)) & (rdf.category == category)]
#         filtered_mdf = mdf[(mdf.year == int(year)) & (mdf.category == category)]
        
        for dim in plane_dimensions:
            column_name = f'{year}_{category}_{dim}'
            column_data = list(filtered_rdf[dim])
            column = Column(column_data, column_name)
            columns.append(column)
            grid_data[column_name] = list(filtered_rdf[dim])
             
        
#         column_name = f'{year}_{category}_{size_dimension}'
#         column_data = list(filtered_rdf[size_dimension])
#         column = Column(column_data, column_name)
#         columns.append(column)
        
        column_name = f'{year}_{category}_codes'
        column_data = list(filtered_rdf['code'])
        column = Column(column_data, column_name)
        columns.append(column)
        
grid = Grid(columns)
py.grid_ops.upload(grid, 'animated_variables_grid')

'https://plot.ly/~sergiochumacero/42/'

#### Creating the Figure

With the grid in place, it's time to define the figure that will be used for generating the plot. Plotly is *declarative* in nature, which basically means that all we need to do is specify what we want, feed the data and let plotly work its magic. Tools of the declarative type have the advantage of being very high level, thus doing a lot of work for us, but for this to happen we have to familiarize ourselves with the specifics of the system at hand in contrast to lower level tools which allow for more flexibility.

The basic structure of the a figure is the following:

In [117]:
figure = {
    'data': [],
    'frames': [],
    'layout': {},
}

A figure is a dictionary where:

The key **data** is a list of dictionaries with references to the values for the x and y dimensions. It also contains information about the type of the data (in our case 'markers') as well as some styling information.

In [118]:
custom_colors = {
    'A': 'rgb(171, 99, 250)',
    'B': 'rgb(230, 99, 250)',
    'C': 'rgb(99, 110, 250)',
    'D': 'rgb(25, 211, 243)',
#     'Oceania': 'rgb(50, 170, 255)'
}


for category in categories:
    data_dict = {
        'xsrc': grid.get_column_reference(f'2014_{category}_{plane_dimensions[0]}'),
        'ysrc': grid.get_column_reference(f'2014_{category}_{plane_dimensions[1]}'),
        'mode': 'markers',
        'marker': {'color': custom_colors[category]},
        'marker': {
            'sizemode': 'area',
            'sizeref': 500,
            'sizesrc': grid.get_column_reference(f'2014_{category}_{size_dimension}'),
            'color': custom_colors[category],
        },
        'textsrc': grid.get_column_reference(f'2014_{category}_codes'),
        'name': 'Categoría: ' + category,
    }
    figure['data'].append(data_dict)

The key **frames** is a list of dictionaries, each one similar to a 'data' dictionary. These are going to be the steps of the animation.

In [119]:
def create_frame(year):
    frame_data = []
    for category in categories:
        year_cat_dict = {
            'xsrc': grid.get_column_reference(f'{year}_{category}_{plane_dimensions[0]}'),
            'ysrc': grid.get_column_reference(f'{year}_{category}_{plane_dimensions[1]}'),
            'mode': 'markers',
            'marker': {
                'sizemode': 'area',
                'sizeref': 500,
                'sizesrc': grid.get_column_reference(f'{year}_{category}_{size_dimension}'),
                'color': custom_colors[category],
            },
            'textsrc': grid.get_column_reference(f'{year}_{category}_codes'),
            'name': 'Categoría: ' + category,
        }
        frame_data.append(year_cat_dict)
        
    return dict(data=frame_data, name=year)

figure['frames'] = [create_frame(year) for year in years]

Finally, the key **layout** is a dictonary specifying different layout settings like title, ranges, labels and buttons.

In [120]:
animation_settings = dict(
    frame={'duration': 700, 'redraw': False},
    fromcurrent=True,
    transition={'duration': 700, 'easing':'cubic-in-out'},
)

steps = []
for year in years:
    step = dict(
        method = 'animate',  
        args = [[year], animation_settings],
        label= year
    )
    steps.append(step)

figure['layout']['sliders'] = [dict(
    active = 0,
    currentvalue = {
        'prefix': 'Año: ',
        'font': {'size': 20},
        'visible': True,
        'xanchor': 'right'
    },
    steps = steps,
    yanchor= 'top',
    xanchor= 'left',
    pad= {'b': 10, 't': 50},
    len= 0.9,
    x= 0.1,
    y= 0,
)]

figure['layout']['updatemenus'] = [{
    'buttons': [{
        'args': [years, animation_settings],
        'label': 'Animar',
        'method': 'animate',
    }],
    'direction': 'left',
    'pad': {'r': 10, 't': 87},
    'showactive': False,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': 0,
    'yanchor': 'top'
}]

x_ind = plane_dimensions[0][1:]
y_ind = plane_dimensions[1][1:]

figure['layout']['hovermode'] = 'closest'
figure['layout']['xaxis'] = {'title': f'Variable {x_ind}: {vdf[vdf.var_id==int(x_ind)]["name"].iloc[0]} ({vdf[vdf.var_id==int(x_ind)]["unit"].iloc[0]})'}
figure['layout']['yaxis'] = {'title': f'Variable {y_ind}: {vdf[vdf.var_id==int(y_ind)]["name"].iloc[0]} ({vdf[vdf.var_id==int(y_ind)]["unit"].iloc[0]})'}
figure['layout']['height'] = 700
figure['layout']['width'] = 1000
figure['layout']['plot_bgcolor'] = '#dfe8f3'
figure['layout']['dragmode'] = 'lasso'
figure['layout']['title'] = 'Variables a través de los años'

To create the plot online, all we need to do now is run the following command.

In [121]:
import plotly.plotly as py
py.icreate_animations(figure, filename='animated_variables_2')

But before commiting our plot to the cloud, we will create an offline version to tinker around and experiment without worrying about the server API request limit.

The first step for this is to use a raw dictionary instead of a plotly grid.

In [114]:
years = ['2014', '2015', '2016',]
categories = ['A', 'B', 'C', 'D']
plane_dimensions = ['v3', 'v22']
size_dimension = 'v22'


custom_colors = {
    'A': 'rgb(171, 99, 250)',
    'B': 'rgb(230, 99, 250)',
    'C': 'rgb(99, 110, 250)',
    'D': 'rgb(25, 211, 243)',
#     'Oceania': 'rgb(50, 170, 255)'
}

rdf = complete_reports_df
mdf = complete_measurements_df

# columns = []

grid_data = {}

for year in years:
    for category in categories:
        filtered_rdf = rdf[(rdf.year == int(year)) & (rdf.category == category)]
        filtered_mdf = mdf[(mdf.year == int(year)) & (mdf.category == category)]
        
        for dim in plane_dimensions:
            column_name = f'{year}_{category}_{dim}'
            grid_data[column_name] = list(filtered_rdf[dim])
             
        grid_data[f'{year}_{category}_{size_dimension}'] = list(filtered_rdf[var])
        grid_data[f'{year}_{category}_codes'] = list(filtered_rdf['code'])
#             columns.append(column_name)
#             column = Column(column_data, column_name)
#             columns.append(column)
        
# pandas_grid = pd.DataFrame(pandas_grid_data, columns=columns)

In [39]:
list(rdf)

['epsa',
 'url_x',
 'v1',
 'v10',
 'v10_type',
 'v11',
 'v11_type',
 'v12',
 'v12_type',
 'v13',
 'v13_type',
 'v14',
 'v14_type',
 'v15',
 'v15_type',
 'v16',
 'v16_type',
 'v17',
 'v17_type',
 'v18',
 'v18_type',
 'v19',
 'v19_type',
 'v1_type',
 'v2',
 'v20',
 'v20_type',
 'v21',
 'v21_type',
 'v22',
 'v22_type',
 'v23',
 'v23_type',
 'v24',
 'v24_type',
 'v25',
 'v25_type',
 'v26',
 'v26_type',
 'v27',
 'v27_type',
 'v28',
 'v28_type',
 'v29',
 'v29_type',
 'v2_type',
 'v3',
 'v30',
 'v30_type',
 'v31',
 'v31_type',
 'v32',
 'v32_type',
 'v33',
 'v33_type',
 'v34',
 'v34_type',
 'v35',
 'v35_type',
 'v36',
 'v36_type',
 'v37',
 'v37_type',
 'v38',
 'v38_type',
 'v39',
 'v39_type',
 'v3_type',
 'v4',
 'v40',
 'v40_type',
 'v41',
 'v41_type',
 'v42',
 'v42_type',
 'v43',
 'v43_type',
 'v44',
 'v44_type',
 'v45',
 'v45_type',
 'v46',
 'v46_type',
 'v47',
 'v47_type',
 'v48',
 'v48_type',
 'v49',
 'v49_type',
 'v4_type',
 'v5',
 'v50',
 'v50_type',
 'v51',
 'v51_type',
 'v5_type',
 'v6

Now we can modify our figure definition to use the data of the raw dictionary instead of grid references.

In [43]:
import numpy as np

figure = {
    'data': [],
    'frames': [],
    'layout': {},
}

for category in categories:
    data_dict = {
        'x': grid_data[f'2014_{category}_ind8'],
        'y': grid_data[f'2014_{category}_ind9'],
        'mode': 'markers',
        'marker': {
            'color': custom_colors[category]
        },
        'marker': {
            'sizemode': 'area',
            'sizeref': 500,
            'size': grid_data[f'2014_{category}_v22'],
#             'color': custom_colors[category]
        },
    }
    figure['data'].append(data_dict)
    
def create_frame(year):
    frame_data = []
    for category in categories:
        year_cat_dict = {
            'x': grid_data[f'{year}_{category}_ind8'],
            'y': grid_data[f'{year}_{category}_ind9'],
            'mode': 'markers',
            'marker': {
                'sizemode': 'area',
                'sizeref': 500,
                'size': grid_data[f'{year}_{category}_v22'],
                'color': custom_colors[category]
            }
        }
        frame_data.append(year_cat_dict)
        
    return dict(data=frame_data)

figure['frames'] = [create_frame(year) for year in years]


figure['layout'] = dict(
    title='Variables a través de los años',
    
#     sliders= [{
#         'args': [
#             'transition', {
#                 'duration': 400,
#                 'easing': 'cubic-in-out'
#             }
#         ],
#         'initialvalue': '2014',
#         'plotlycommand': 'animate',
#         'values': years,
#         'visible': True,
#     }],
    
    updatemenus= [{
        'buttons': [{
            'args': [None],
            'label': 'Play',
            'method': 'animate',
        }, {
            'args': [None],
            'label': 'Pause',
            'method': 'animate',
        }],
        'showactive': False,
        'type': 'buttons',
    }]
)

figure['layout']['sliders'] = [dict()]

# figure['layout']['sliders'] = [dict(
#     visible=True,
#     active=0,
#     yanchor='top',
#     xanchor='left',
#     currentvalue={
#         'font': {'size':20},
#         'prefix': 'text-before',
#         'visible': True,
#         'xanchor': 'right',
#     },
#     transition={
#         'duration': 300, 'easing': 'cubic-in-out'},
#     pad={
#         'b': 10,
#         't': 50,
#     },
#     steps=[dict(
#         method='animate',
#         label='some-label',
#         value='2014',
#         args=[{
#             'frame': {'duration':300, 'redraw': False},
#             'mode': 'immediate',
#         }]
#     )]
# )]

# {
#     'method': 'animate',
#     'label': 'label-for-frame',
#     'value': 'value-for-frame(defaults to label)',
#     'args': [{'frame': {'duration': 300, 'redraw': False},
#          'mode': 'immediate'}
#     ],
# }

# figure['layout']['sliders] = 
#     'active': 0,
#     'yanchor': 'top',
#     'xanchor': 'left',
#     'currentvalue': {
#         'font': {'size': 20},
#         'prefix': 'text-before-value-on-display',
#         'visible': True,
#         'xanchor': 'right'
#     },
#     'transition': {'duration': 300, 'easing': 'cubic-in-out'},
#     'pad': {'b': 10, 't': 50},
#     'len': 0.9,
#     'x': 0.1,
#     'y': 0,
# #     'steps': [...]


Now we can plot our animated plot using 'iplot' offline! Success! Now we can tinker with styling all we want.

In [44]:
iplot(figure)

In [55]:
variables_r = requests.get('https://peridash.ml/api/variables')
variables_df = pd.read_json(variables_r.text)

In [71]:
vdf = variables_df

# vdf[vdf.var_id==3].name.iloc[0]

vdf[vdf.var_id==3].unit.iloc[0]

'm3/periodo'

In [115]:
import numpy as np
from plotly.offline import iplot
# import plotly.plotly as py


figure = dict(data=[], layout={}, frames=[])
animation_settings = dict(
    frame={'duration': 700, 'redraw': False},
    fromcurrent=True,
    transition={'duration': 700, 'easing':'cubic-in-out'},
)

for category in categories:
    data_dict = {
        'x': grid_data[f'2014_{category}_{plane_dimensions[0]}'],
        'y': grid_data[f'2014_{category}_{plane_dimensions[1]}'],
        'mode': 'markers',
        'marker': {
            'color': custom_colors[category]
        },
        'marker': {
            'sizemode': 'area',
            'sizeref': 500,
            'size': grid_data[f'2014_{category}_{size_dimension}'],
            'color': custom_colors[category],
        },
        'text': grid_data[f'2014_{category}_codes'],
        'name': 'Categoría: ' + category,
    }
    figure['data'].append(data_dict)

steps = []
for year in years:
    step = dict(
        method = 'animate',  
        args = [[year], animation_settings],
        label= year
    )
    steps.append(step)

figure['layout']['sliders'] = [dict(
    active = 0,
    currentvalue = {
        'prefix': 'Año: ',
        'font': {'size': 20},
        'visible': True,
        'xanchor': 'right'
    },
    steps = steps,
    yanchor= 'top',
    xanchor= 'left',
    pad= {'b': 10, 't': 50},
    len= 0.9,
    x= 0.1,
    y= 0,
)]

figure['layout']['updatemenus'] = [{
    'buttons': [{
        'args': [years, animation_settings],
        'label': 'Animar',
        'method': 'animate',
    }],
    'direction': 'left',
    'pad': {'r': 10, 't': 87},
    'showactive': False,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': 0,
    'yanchor': 'top'
}]

x_ind = plane_dimensions[0][1:]
y_ind = plane_dimensions[1][1:]

figure['layout']['hovermode'] = 'closest'
figure['layout']['xaxis'] = {'title': f'Variable {x_ind}: {vdf[vdf.var_id==int(x_ind)]["name"].iloc[0]} ({vdf[vdf.var_id==int(x_ind)]["unit"].iloc[0]})'}
figure['layout']['yaxis'] = {'title': f'Variable {y_ind}: {vdf[vdf.var_id==int(y_ind)]["name"].iloc[0]} ({vdf[vdf.var_id==int(y_ind)]["unit"].iloc[0]})'}
figure['layout']['height'] = 700
figure['layout']['width'] = 1000
figure['layout']['plot_bgcolor'] = '#dfe8f3'
figure['layout']['dragmode'] = 'lasso'

def create_frame(year):
    frame_data = []
    for category in categories:
        year_cat_dict = {
            'x': grid_data[f'{year}_{category}_{plane_dimensions[0]}'],
            'y': grid_data[f'{year}_{category}_{plane_dimensions[1]}'],
            'mode': 'markers',
            'marker': {
                'sizemode': 'area',
                'sizeref': 500,
                'size': grid_data[f'{year}_{category}_{size_dimension}'],
                'color': custom_colors[category],
            },
            'text': grid_data[f'{year}_{category}_codes'],
            'name': 'Categoría: ' + category,
        }
        frame_data.append(year_cat_dict)
        
    return dict(data=frame_data, name=year)

figure['frames'] = [create_frame(year) for year in years]


iplot(figure)